In [283]:
from app.database import get_db, get_opensearch
from sqlalchemy import text, Table, Column, MetaData, Text, bindparam
from sqlalchemy.dialects.postgresql import UUID as UUID_column_type
import json
import pandas as pd


In [326]:
             #                   """
             # SELECT *
             #    FROM custom_terminologies.code
             #    WHERE terminology_version_uuid = '50ed5f2c-7c4c-4586-a7d8-f3665d6380c0'
             #    ORDER BY display DESC;
             #    """

conn = get_db()

In [327]:
data = conn.execute(
            text(
                """
                WITH DuplicateDisplays AS (
                    SELECT display
                    FROM custom_terminologies.code
                    WHERE terminology_version_uuid = '50ed5f2c-7c4c-4586-a7d8-f3665d6380c0'
                    GROUP BY display
                    HAVING COUNT(*) > 1
                ), RelevantDisplays AS (
                    SELECT DISTINCT display
                    FROM custom_terminologies.code
                    WHERE terminology_version_uuid = '50ed5f2c-7c4c-4586-a7d8-f3665d6380c0'
                      AND code LIKE '%code%'
                      AND display IN (SELECT display FROM DuplicateDisplays)
                )
                SELECT c.*
                FROM custom_terminologies.code c
                INNER JOIN RelevantDisplays rd ON c.display = rd.display
                WHERE c.terminology_version_uuid = '50ed5f2c-7c4c-4586-a7d8-f3665d6380c0'
                ORDER BY c.display DESC;
                """
            ),
            {"concept_map_version_uuid": '50ed5f2c-7c4c-4586-a7d8-f3665d6380c0'},
        )

In [328]:
conn.close()

In [329]:
type(data)

sqlalchemy.engine.cursor.CursorResult

In [330]:
df = pd.DataFrame(data.fetchall(), columns=data.keys())

In [331]:
df

,uuid,code,display,terminology_version_uuid,additional_data,depends_on_property,depends_on_system,depends_on_value,depends_on_display,created_date
0,fb1105a7-c8b9-4461-89cd-5bc9f60d8364,"{[{14040001000004109, Uncontrolled pain (finding), http://snomed.info/sct}, {R52, Uncontrolled pain, urn:oid:2.16.840.1.113883.6.90}], Uncontrolled pain}",Uncontrolled pain,50ed5f2c-7c4c-4586-a7d8-f3665d6380c0,"{'automapping_target': '14040001000004109|Uncontrolled pain (finding)', 'automapping_evidence': 'Condition Code.text is an exact match for SNOMED PT', 'automapping_relationship': 'Equivalent', 'count_of_resources_affected': '1330'}",,,,,2023-03-01 00:00:00+00:00
1,b4646298-292f-4d21-8bd8-f9c0166c1fd8,"{""coding"": [{""system"": ""http://hl7.org/fhir/sid/icd-10-cm"", ""code"": ""R52""}, {""system"": ""http://snomed.info/sct"", ""code"": ""14040001000004109""}], ""text"": ""Uncontrolled pain""}",Uncontrolled pain,50ed5f2c-7c4c-4586-a7d8-f3665d6380c0,None,,,,,2023-12-07 15:10:13.719826+00:00
2,cd970f63-99bb-47f8-a298-72025f6ede90,"{[{1089991000119109, Oral ulcerative mucositis due to antineoplastic therapy (disorder), http://snomed.info/sct}, {528.01, Ulcerative oral mucositis due to antineoplastic therapy, http://hl7.org/fhir/sid/icd-9-cm/diagnosis}, {K12.31, Ulcerative oral mucositis due to antineoplastic therapy, urn:oid:2.16.840.1.113883.6.90}], Ulcerative oral mucositis due to antineoplastic therapy}",Ulcerative oral mucositis due to antineoplastic therapy,50ed5f2c-7c4c-4586-a7d8-f3665d6380c0,"{'automapping_target': None, 'automapping_evidence': None, 'automapping_relationship': None, 'count_of_resources_affected': '3341'}",,,,,2023-03-01 00:00:00+00:00
3,25e0a8d9-e93e-43fc-897c-be05578bdf1e,"{[{K12.31, null, urn:oid:2.16.840.1.113883.6.90}, {1089991000119109, null, urn:oid:2.16.840.1.113883.6.96}], Ulcerative oral mucositis due to antineoplastic therapy}",Ulcerative oral mucositis due to antineoplastic therapy,50ed5f2c-7c4c-4586-a7d8-f3665d6380c0,"{'automapping_target': None, 'automapping_evidence': None, 'automapping_relationship': None, 'count_of_resources_affected': '306'}",,,,,2023-03-01 00:00:00+00:00
4,9ff86800-6c49-4f03-98ea-c648df4c55b0,"{""coding"": [{""system"": ""http://hl7.org/fhir/sid/icd-9-cm/diagnosis"", ""code"": ""528.01"", ""display"": ""Ulcerative oral mucositis due to antineoplastic therapy""}, {""system"": ""http://hl7.org/fhir/sid/icd-10-cm"", ""code"": ""K12.31"", ""display"": ""Ulcerative oral mucositis due to antineoplastic therapy""}], ""text"": ""Ulcerative oral mucositis due to antineoplastic therapy""}",Ulcerative oral mucositis due to antineoplastic therapy,50ed5f2c-7c4c-4586-a7d8-f3665d6380c0,None,,,,,2023-12-07 15:10:13.719826+00:00
...,...,...,...,...,...,...,...,...,...,...
420,212038a2-f811-423c-9d53-24f2a56b35e4,"{[{11381005, Acne (disorder), http://snomed.info/sct}, {706.1, Acne, not otherwise specified, http://hl7.org/fhir/sid/icd-9-cm/diagnosis}, {L70.9, Acne, not otherwise specified, urn:oid:2.16.840.1.113883.6.90}], Acne, not otherwise specified}","Acne, not otherwise specified",50ed5f2c-7c4c-4586-a7d8-f3665d6380c0,"{'automapping_target': None, 'automapping_evidence': None, 'automapping_relationship': None, 'count_of_resources_affected': '133'}",,,,,2023-03-01 00:00:00+00:00
421,ea8b4079-119d-4254-926a-8b55f5778076,"{""coding"": [{""system"": ""http://snomed.info/sct"", ""code"": ""11381005"", ""display"": ""Acne (disorder)""}, {""system"": ""http://hl7.org/fhir/sid/icd-9-cm/diagnosis"", ""code"": ""706.1"", ""display"": ""Acne, not otherwise specified""}, {""system"": ""http://hl7.org/fhir/sid/icd-10-cm"", ""code"": ""L70.9"", ""display"": ""Acne, not otherwise specified""}], ""text"": ""Acne, not otherwise specified""}","Acne, not otherwise specified",50ed5f2c-7c4c-4586-a7d8-f3665d6380c0,None,,,,,2023-12-07 15:07:57.139917+00:00
422,3a9c520e-4e4f-4b1e-9af3-f7f9c90ab1ce,"{[{274533004, Abnormal findings on diagnostic imaging of lung (finding), http://snomed.info/sct}, {793.19, Abnormal finding

In [184]:
import datetime
import json
import re
from enum import Enum

from app.helpers.data_helper import is_uuid_format, is_spark_format, is_json_format, load_json_string, \
    normalize_source_ratio, serialize_json_object, escape_for_sql, normalize_source_codeable_concept, \
    cleanup_json_string
from app.helpers.message_helper import message_exception_classname


class DataExtensionUrl(Enum):
    """
    DataExtensionUrls for ConceptMaps from the RCDM specification
    """
    SOURCE_CODEABLE_CONCEPT = "http://projectronin.io/fhir/StructureDefinition/ronin-conceptMapSourceCodeableConcept"
    SOURCE_RATIO = "http://projectronin.io/fhir/StructureDefinition/ronin-conceptMapSourceRatio"


class IssuePrefix(Enum):
    """
    Prefix for messages reporting format issues
    """
    COLUMN_VALUE_FORMAT = "format issue: "


def convert_string_to_datetime_or_none(input_string):
    if input_string is None:
        return None
    else:
        try:
            return datetime.datetime.strptime(input_string, "%Y-%m-%dT%H:%M:%S.%fZ")
        except ValueError:
            return datetime.datetime.strptime(input_string, "%Y-%m-%dT%H:%M:%SZ")


def prepare_dynamic_value_for_sql(old_value: str, old_display: str = None) -> (str, str, str, str):
    """
    For inserting a code value into a table that has already been migrated to the ConceptMap v5 schema.

    Calls the helper function prepare_dynamic_value_for_sql_issue() to process the inputs.

    @param old_value - may be a string, or may be a FHIR code which as data is a string value
    @param old_display - IMPORTANT - caller must supply this value if the old_value is a FHIR code - used for some tests
    @raise ValueError if the old_value cannot be correctly stored, so the caller should not insert it into the database.
    """
    (value_schema, value_simple, value_jsonb, value_string) = prepare_dynamic_value_for_sql(old_value, old_display)
    if value_schema is None:
        raise ValueError("Value cannot be stored: format issues found")
    if IssuePrefix.COLUMN_VALUE_FORMAT.value in value_schema:
        raise ValueError(value_schema)
    return value_schema, value_simple, value_jsonb, value_string


def prepare_dynamic_value_for_sql_issue(old_value: str, old_display: str = None) -> (str, str, str, str):
    """
    Converts 1 input string value into 4 strings to use in forming correct syntax for a subsequent SQL INSERT query.
    This function is intended to support multiple data types for FHIR dynamic values: code, CodeableConcept, Ratio, etc.

    For CodeableConcept only, old_display (the associated display value) may be provided and an extra error check
    happens. For other FHIR dynamic value data types, the old_display is ignored.

    @param old_value - may be a string, or may be a FHIR code which as data is a string value
    @param old_display - IMPORTANT - caller must supply this value if the old_value is a FHIR code - used for some tests

    @return
        value_schema (string) - FHIR primitive name, DataExtensionUrls value, or message prefixed with "format issue:"
        value_simple (string) - old_code if primitive or old_code has "format issue: " - else None
        value_jsonb (string) - None if primitive or old_value has "format issue:" - else, object with that value_schema
            - note: because this function prepares these values for SQL INSERT query formation, this value is a string -
            all JSON uses double quotes, and if there is a single quote value in the JSON, it is '' for SQL escaping
        value_string (string) - either value_simple, or value_jsonb binary JSON value, correctly serialized to a string
            - this returned value_string value is ready for the  caller to input to hash_string to create the code_id
    """
    # init
    value_schema = None
    value_simple = None
    value_jsonb = None
    value_string = None

    # unwrap any wrappers in the JSON
    if old_value is not None and '"valueCodeableConcept"' in old_value:
        old_value = remove_deprecated_wrapper(old_value)

    # begin: detect code issue cases
    # handle None (bad)
    if old_value is None:
        (value_schema, value_simple, value_jsonb, value_string) = prepare_code_none_issue_for_sql(
            issue="value is None",
            id="value is null"
        )

    # handle '' (bad if this is a code; fine for string)
    elif old_display is not None and old_value == "":
        (value_schema, value_simple, value_jsonb, value_string) = prepare_string_format_issue_for_sql(
            issue="value is ''",
            code_string=old_value
        )

    # handle uuid case (bad)
    elif is_uuid_format(old_value):
        (value_schema, value_simple, value_jsonb, value_string) = prepare_string_format_issue_for_sql(
            issue="uuid",
            code_string=old_value
        )

    # handle '[]' (bad)
    elif old_value == "[]":
        (value_schema, value_simple, value_jsonb, value_string) = prepare_string_format_issue_for_sql(
            issue="value is '[]'",
            code_string=old_value
        )

    # handle '{}' (bad)
    elif old_value == "{}":
        (value_schema, value_simple, value_jsonb, value_string) = prepare_string_format_issue_for_sql(
            issue="value is " + "'{}'",
            code_string=old_value
        )

    # handle '[null]' (bad)
    elif old_value == "[null]":
        (value_schema, value_simple, value_jsonb, value_string) = prepare_string_format_issue_for_sql(
            issue="value is '[null]'",
            code_string=old_value
        )

    # handle 'null' (bad)
    elif old_value == "null":
        (value_schema, value_simple, value_jsonb, value_string) = prepare_string_format_issue_for_sql(
            issue="value is 'null'",
            code_string=old_value
        )

    # handle {"text":"Line 1"} spark or JSON (bad)
    elif "{Line 1}" in old_value or ('"text"' in old_value and '"Line 1"' in old_value):
        (value_schema, value_simple, value_jsonb, value_string) = prepare_string_format_issue_for_sql(
            issue='{"text":"Line 1"}',
            code_string=old_value
        )

    # handle coding not present in CodeableConcept (bad)
    elif '"text"' in old_value and '"coding"' not in old_value:
        (value_schema, value_simple, value_jsonb,value_string) = prepare_string_format_issue_for_sql(
            issue="CodeableConcept with only text",
            code_string=old_value
        )

    # handle spark case (bad)
    elif is_spark_format(old_value):
        # try to convert to json
        json_string = convert_source_concept_spark_export_string_to_json_string(old_value)
        # failed to convert to json: report format issue
        if json_string == old_value:
            (value_schema, value_simple, value_jsonb, value_string) = prepare_string_format_issue_for_sql(
                issue="spark",
                code_string=old_value
            )
        # converted to json: handle the json
        else:
            (value_schema, value_simple, value_jsonb, value_string) = prepare_json_format_for_sql(json_string)

    # handle json case (good or bad)
    elif is_json_format(old_value):
        (value_schema, value_simple, value_jsonb, value_string) = prepare_json_format_for_sql(old_value)

    # see if code and display columns were reversed
    elif old_display is not None and is_code_display_reversed(old_value, old_display):
        if value_schema is not None and len(value_schema) > 0:
            value_schema += ", code and display might be reversed"
        else:
            (value_schema, value_simple, value_jsonb, value_string) = prepare_string_format_issue_for_sql(
                issue="code and display might be reversed",
                code_string=old_value
            )

    # string case (good) - if there is no old_display supplied by the caller, it is not a code: it is a depends_on
    elif old_display is None:
        (value_schema, value_simple, value_jsonb, value_string) = prepare_string_depends_on_for_sql(
            code_string=old_value
        )

    # handle code case (good)
    else:
        (value_schema, value_simple, value_jsonb, value_string) = prepare_string_source_code_for_sql(
            code_string=old_value
        )
    # end: detect code issue cases

    return value_schema, value_simple, value_jsonb, value_string


def prepare_json_format_for_sql(json_string: str) -> (str, str, str, str):
    """
    Normalizes the JSON string to an expected format with double quotes and no wasted space - calls helper functions
    in the correct order: load the object, normalize to RCDM (sorts lists), serialize (sorts keys, strips space)
    @return
        code_schema (string) - FHIR primitive name, DataExtensionUrls value, or message prefixed with "format issue:"
        code_simple (string) - old_code if primitive or old_code has "format issue: " - else None
        code_jsonb (string) - None if primitive or old_code has "format issue:" - else, the object with that code_schema
            - note: because this function prepares these values for SQL INSERT query formation, this value is a string -
            all JSON uses double quotes, and if there is a single quote value in the JSON, it is '' for SQL escaping
        code_string (string) - either code_simple, or the code_jsonb binary JSON value correctly serialized to a string
    """
    try:
        json_object = load_json_string(json_string)

        # Ratio
        if '"numerator"' in json_string or '"denominator"' in json_string:
            # todo: prepare_object_source_ratio_for_sql(ratio_object=json_object) - not yet, questions out to Content
            (code_schema, code_simple, code_jsonb, code_string) = prepare_object_format_issue_for_sql(
                issue="Ratio needs to be supported",
                code_string=json_string
            )

        # CodeableConcept
        elif '"text"' in json_string or (
                '"coding"' in json_string and (
                '"code"' in json_string
                or '"display"' in json_string
                or '"system"' in json_string
                or '"version"' in json_string
        )
        ):
            try:
                (code_schema, code_simple, code_jsonb, code_string) = prepare_object_source_code_for_sql(
                    code_object=json_object
                )
            except Exception as e:
                name = message_exception_classname(e)
                if name == "BadDataError":
                    name = "invalid JSON for CodeableConcept"
                (code_schema, code_simple, code_jsonb, code_string) = prepare_object_format_issue_for_sql(
                    issue=name,
                    code_string=json_string
                )

        # unsupported FHIR resource type, or an unexpected data dictionary
        else:
            (code_schema, code_simple, code_jsonb, code_string) = prepare_object_format_issue_for_sql(
                issue="JSON is an unsupported FHIR resource type, or an unexpected data dictionary",
                code_string=json_string
            )

    except Exception as e:
        name = message_exception_classname(e)
        if name == "JSONDecodeError":
            name = "invalid JSON for json.loads()"
        (code_schema, code_simple, code_jsonb, code_string) = prepare_string_format_issue_for_sql(
            issue=name,
            code_string=json_string
        )

    return code_schema, code_simple, code_jsonb, code_string


def prepare_object_source_ratio_for_sql(code_object) -> (str, str, str, str):
    """
    @throws BadDataError if the value does not match the sourceRatio schema
    """
    rcdm_object = normalize_source_ratio(code_object)
    rcdm_string = serialize_json_object(rcdm_object)
    return (
        DataExtensionUrl.SOURCE_RATIO.value,
        None,
        escape_for_sql(rcdm_string),
        rcdm_string
    )


def prepare_object_source_code_for_sql(code_object) -> (str, str, str, str):
    """
    @throws BadDataError if the value does not match the sourceCodeableConcept schema
    """
    rcdm_object = normalize_source_codeable_concept(code_object)
    rcdm_string = serialize_json_object(rcdm_object)
    return (
        DataExtensionUrl.SOURCE_CODEABLE_CONCEPT.value,
        None,
        escape_for_sql(rcdm_string),
        rcdm_string
    )


def prepare_data_dictionary_for_sql(info_dict) -> str:
    """
    Serialize a python data dictionary for a string valued column - like additional_data for INFX internal use only
    """
    if info_dict is None:
        return None
    info_string = json.dumps(info_dict)
    clean_info = cleanup_json_string(info_string)
    sql_escaped = escape_for_sql(clean_info)
    return sql_escaped


def prepare_string_depends_on_for_sql(code_string: str) -> (str, str, str, str):
    if code_string is None or code_string == "":
        return (
            None,
            None,
            None,
            None
        )
    else:
        return (
            "string",
            code_string,
            None,
            code_string
        )


def prepare_string_source_code_for_sql(code_string: str) -> (str, str, str, str):
    return (
        "code",
        code_string,
        None,
        code_string
    )


def prepare_code_none_issue_for_sql(issue: str, id: str) -> (str, str, str, str):
    return (
        f"{IssuePrefix.COLUMN_VALUE_FORMAT.value}{issue}",
        None,
        None,
        id
    )


def prepare_string_format_issue_for_sql(issue: str, code_string: str) -> (str, str, str, str):
    return (
        f"{IssuePrefix.COLUMN_VALUE_FORMAT.value}{issue}",
        code_string,
        None,
        code_string
    )


def append_string_format_issue_for_sql(issue: str, schema: str) -> str:
    if schema is None or len(schema) == 0:
        return f"{IssuePrefix.COLUMN_VALUE_FORMAT.value}{issue}"
    else:
        return schema + f", {issue}"


def prepare_object_format_issue_for_sql(issue: str, code_string: str) -> (str, str, str, str):
    code_clean = cleanup_json_string(code_string)
    return (
        f"{IssuePrefix.COLUMN_VALUE_FORMAT.value}{issue}",
        code_clean,
        None,
        code_clean
    )


def remove_deprecated_wrapper(input_string: str) -> str:
    """
    deprecated wrapper convention uses keys "component", "valueCodeableConcept", or both - remove if present
    @return (bool: whether the wrapper was found and removed, str: the string to use after unwrapping)
    """
    output_string = input_string
    if is_json_format(input_string):
        try:
            json_object = load_json_string(input_string)
            component = json_object.get("component")
            if component is None:
                codeable_concept = json_object.get("valueCodeableConcept")
                if codeable_concept is not None:
                    output_string = serialize_json_object(codeable_concept)
            else:
                if isinstance(component, list):
                    # It is only safe to grab the valueCodeableConcept for this repair if it is single
                    if len(component) == 1:
                        for concept in component:
                            codeable_concept = concept.get("valueCodeableConcept")
                            if codeable_concept is not None:
                                output_string = serialize_json_object(codeable_concept)
                else:
                    codeable_concept = component.get("valueCodeableConcept")
                    if codeable_concept is not None:
                        output_string = serialize_json_object(codeable_concept)
        except Exception as e:
            pass
    return output_string


def is_code_display_reversed(code_string: str, display_string: str):
    """
    good-enough check for likelihood that this is a case where code and display columns were reversed by a load function
    """
    # In valid code/display pairs it is unlikely that code and display have the same value or that either is empty
    if code_string is None:
        if display_string is None:
            return False
        else:
            return True
    if code_string == display_string:
        return False

    # Check length 0
    code_len = len(code_string)
    if display_string is None:
        display_len = 0
    else:
        display_len = len(display_string)
    if code_len == 0 and display_len > 0:
        return True
    if display_len == 0 and code_len > 0:
        return False

    code_alpha_count = sum(1 for char in code_string if char.isalpha())
    code_digit_count = sum(1 for char in code_string if char.isdigit())
    code_hyphen_count = sum(1 for char in code_string if char == "-")
    code_alnum_count = code_alpha_count + code_digit_count
    expected_code_len = code_alnum_count + code_hyphen_count

    display_alpha_count = sum(1 for char in display_string if char.isalpha())
    display_digit_count = sum(1 for char in display_string if char.isdigit())
    display_hyphen_count = sum(1 for char in display_string if char == "-")
    display_alnum_count = display_alpha_count + display_digit_count
    expected_display_code_len = display_alnum_count + display_hyphen_count

    # There are non-typical values in the code value, while at the same time, display looks like a code value
    if code_len > expected_code_len and display_len == expected_display_code_len:
        return True
    # There are more alphabetic values in the code than in the display
    if code_alpha_count > display_alpha_count:
        return True
    # There are more numeric values in the display than in the code
    if code_digit_count < display_digit_count:
        return True
    # There are too many numeric values in the display for it to be human-readable
    if display_digit_count == display_len or (display_digit_count >= (display_len / 2)):
        return True
    # If there are only alnum or hyphen characters in the code, and it is reasonably short, it is probably fine
    if code_len == expected_code_len and code_len <= display_len:
        return False
    # In FHIR code/display value sets, display has (title case) + spaces, while code is (lowercase or digits) + hyphens
    code_clean = "".join(code_string.split("-"))
    display_clean = "".join(display_string.split(" "))
    if (
        len(code_clean) == len(display_clean)
            and code_clean[0].isupper()
            and display_clean[0].islower()
            and code_clean[0].lower() == display_clean[0]
    ):
        return True
    # We could not identify any issues with this code/display pair
    return False


def convert_source_concept_spark_export_string_to_json_string(spark_export_string: str) -> str:
    """
    @raise exception if parsing hits any snags, which it should not
    """
    # init
    json_object: dict = {}
    coding_list = []
    value_list = re.split(r'], ', spark_export_string)

    # coding
    if len(value_list) > 0:
        coding_string = value_list[0][2:]
        coding_value_list = re.split(r'}, \{', coding_string)
        if len(coding_value_list) > 0:
            coding_value_list[0] = coding_value_list[0][1:]
            coding_value_list[-1] = coding_value_list[-1][:-1]
            for coding_value in coding_value_list:
                attribute = re.split(r', ', coding_value)
                coding_object: dict = {}
                if len(attribute) >= 3:
                    first = attribute[0]
                    middle = ", ".join(attribute[1:-1])
                    last = attribute[-1]
                    if first != "null":
                        coding_object.update({"code": first})
                    if middle != "null":
                        coding_object.update({"display": middle})
                    if last != "null":
                        coding_object.update({"system": last})
                elif len(attribute) == 2:
                    first = attribute[0]
                    last = attribute[-1]
                    if first != "null":
                        coding_object.update({"code": first})
                    if last != "null":
                        coding_object.update({"system": last})
                else:
                    return spark_export_string
                if len(coding_object) > 0:
                    coding_list.append(coding_object)
            if len(coding_list) > 0:
                json_object.update({"coding": coding_list})
        else:
            return spark_export_string
    else:
        return spark_export_string

    # text
    if len(value_list) > 1:
        text_value = value_list[1][:-1]
        if text_value != "null":
            json_object.update({"text": text_value})

    # done
    return serialize_json_object(json_object)


In [226]:
import hashlib
import json

import uuid
from collections import OrderedDict

from app.errors import BadDataError


def contains_double_quoted_strings(input_string: str) -> bool:
    """
    good-enough check for at least 1 pair of double quotes
    """
    return sum(1 for char in input_string if char == '"') >= 2


def contains_single_quoted_strings(input_string: str) -> bool:
    """
    good-enough check for at least 1 pair of single quotes
    """
    return sum(1 for char in input_string if char == "'") >= 2


def contains_colon(input_string: str) -> bool:
    """
    good-enough check for at least 1 colon
    """
    return ":" in input_string


def contains_braces(input_string: str) -> bool:
    """
    good-enough check for curly braces
    """
    return "{" in input_string and "}" in input_string


def contains_brackets(input_string: str) -> bool:
    """
    good-enough check for square brackets
    """
    return "[" in input_string and "]" in input_string


def contains_alphanumeric(input_string: str):
    return any(char.isalnum() for char in input_string)


def get_next_hex_char(hex_char):
    hex_int = int(hex_char, 16)
    hex_int = (hex_int + 1) % 16
    next_hex_char = hex(hex_int)[2:]
    return next_hex_char


def load_json_string(input_json_string: str):
    """
    json.loads() options:
    (no options explicitly provided at present - uses defaults)
    """
    return json.loads(input_json_string)


def cleanup_json_string(input_json_string: str) -> str:
    """
    Shortcut to ensure consistent JSON by calling load_json_string() then serialize_json_object().
    This function is quick and basic and respects existing list member order.

    If you want to force lists in input_json_string into order using order_object_List(),
    do not use this function; instead use one of the following sequences:

    1. Ordinary JSON:
    Call load_json_string(),
    call order_object_list() on any lists, then
    call serialize_json_object().

    2. RCDM profile JSON:
    Call load_json_string(),
    provide and call a normalization function like normalize_source_codeable_concept() to prune the object attributes,
    call order_object_list() on any lists, then
    call serialize_json_object().
    """
    json_object = load_json_string(input_json_string)
    return serialize_json_object(json_object)


def serialize_json_object(json_object) -> str:
    """
    If the json_object contains lists, and you want to force list members into a known, consistent order, do not start
    with this function; first call order_object_list() on any lists, then call serialize_json_object() on that result
    json.dumps() options:
    sort_keys=True - sort the keys in the output string alphabetically; respects the current order of list elements
    ensure_ascii=False - ensures that double quotes are used to wrap strings in the output, needed for SQL query format
    separators=(",", ":") - omits space characters from JSON syntax, but retains space characters inside JSON content
    """
    return json.dumps(json_object, sort_keys=True, ensure_ascii=False, separators=(",", ":"))


def is_json_format(input_string: str) -> bool:
    """
    good-enough check for json punctuation to format at least one key-value pair in the object
    """
    clean_string = "".join(input_string.split(" "))
    return (
        contains_brackets(clean_string) or contains_braces(clean_string)
    ) and (
        contains_colon(clean_string)
    ) and ((
        contains_double_quoted_strings(input_string) and '":"' in clean_string
        ) or (
        contains_single_quoted_strings(input_string) and "':'" in clean_string
    ))


def is_spark_format(input_string: str) -> bool:
    """
    good-enough check for spark format
    """
    return (
        contains_braces(input_string) or contains_brackets(input_string)
    ) and (
        contains_alphanumeric(input_string)
    ) and (
        not is_json_format(input_string)
    )


def is_uuid_format(input_string: str) -> bool:
    """
    check for uuid4 format
    """
    try:
        return uuid.UUID(input_string).version == 4
    except ValueError:
        return False


def hash_string(input_string: str) -> str:
    """
    For all hashed id values - like mapping_id and code_id - caller is responsible for a correctly formed input_string
    """
    # create a new md5 hash object
    hash_object = hashlib.md5()
    # update the hash object with the bytes-like object
    hash_object.update(input_string.encode("utf-8"))
    # return the hexadecimal representation of the hash
    return hash_object.hexdigest()


def hash_jsonb(input_object) -> str:
    """
    serialize an object to a string and then hash the string
    """
    return hash_string(serialize_json_object(input_object))


def escape_for_sql(input_string: str) -> str:
    """
    SQL escape single quotes and colons in JSON string content values so they can be input to INSERT - "Hodgkin's: yes"
    """
    # single quote
    escaped_single_quotes = input_string.replace("'", "''")

    # colon
    colon_content_list = escaped_single_quotes.split('":')
    escaped_colon_content_list = []
    for content in colon_content_list:
        escaped = content.replace(".:", ".\:")
        escaped_colon_content_list.append(escaped)
    escaped_string = '":'.join(escaped_colon_content_list)
    return escaped_string

    # todo:
    # So far, there have been issues handling true/false in the JSON string when we get to SQL. This was a formatting
    # issue for our SQL INSERT query text. Text values in JSON work fine with our query because we wrap them in
    # double quotes, but unwrapped values, e.g. boolean, had sqlalchemy formatting errors. For CodeableConcept this
    # was resolved by RCDM not using userSelected, a boolean. Adding Ratio etc, we should beware numeric and boolean.
    # The split('".') used to find colon characters in content could also be handy to find cases like {"mine":true}


def normalize_source_ratio(input_object):
    """
    shape the Ratio attributes as needed to conform to the RCDM profile - remove the unsupported attributes -
    require required attributes
    """
    # todo: implement
    return input_object


def normalize_source_codeable_concept(input_object):
    """
    shape the CodeableConcept attributes as needed to conform to the RCDM profile - remove the unsupported attributes -
    require required attributes - calls order_object_list() on "coding" list to ensure members are in consistent order
    - ≈
    """
    if input_object.get("id") is not None:
        del input_object["id"]
    coding_list = input_object.get("coding")
    text_value = input_object.get("text")
    if coding_list is None and text_value is None:
        raise BadDataError(
            code="CodeableConcept.schema",
            description="CodeableConcept was expected, but the object has no coding or text attribute at the top level",
            errors="Invalid CodeableConcept"
        )
    if coding_list is not None:
        for coding in coding_list:
            if coding.get("id") is not None:
                del coding["id"]
            if coding.get("userSelected") is not None:
                del coding["userSelected"]
        input_object["coding"] = order_object_list(coding_list)
    return input_object


def order_object_list(input_object_list) -> list:
    """
    Provide a consistent, standard order for FHIR array members that are objects. Per the FHIR standard. arrays do not
    have any guaranteed order. Per the json library, arrays cannot be given a standard order by dumps() or loads().

    The technique we use is to do a md5 numeric hash of each list member  and sort list members by their hash values.

    The ONLY requirement is to guarantee we always output any identical list members appear in the SAME ORDER in our
    lists, even if they did not arrive in that same order in the data. There is no requirement to ALSO make this order
    VISIBLE or INTUITIVE to humans. We have no way to make the ordering criteria apparent to human viewers, but it works

    Function steps in detail:
    1. serialize and hash each member of the input_list using our own functions (which standardize key order in JSON)
    2. use the hash values as keys in a dictionary with the values being the corresponding input_list member
    3. create a new list from the dictionary values in hash value key order
    Note: for now, ignore the infinite theoretical potential for additional levels of lists within objects within lists:
    will handle when we encounter an RCDM defined model that permits this and is also in a jsonb column in the database.

    @param input_object_list is a list of objects, such as CodeableConcept.coding list
    @return a new list in an order that will be consistent any time the same set of list members are input in any order
    """
    object_list: dict = {}
    for obj in input_object_list:
        object_list.update({hash_jsonb(obj): obj})
    sorted_list = OrderedDict(sorted(object_list.items()))
    return list(sorted_list.values())


In [332]:
    # @return
    #     value_schema (string) - FHIR primitive name, DataExtensionUrls value, or message prefixed with "format issue:"
    #     value_simple (string) - old_code if primitive or old_code has "format issue: " - else None
    #     value_jsonb (string) - None if primitive or old_value has "format issue:" - else, object with that value_schema
    #         - note: because this function prepares these values for SQL INSERT query formation, this value is a string -
    #         all JSON uses double quotes, and if there is a single quote value in the JSON, it is '' for SQL escaping
    #     value_string (string) - either value_simple, or value_jsonb binary JSON value, correctly serialized to a string
    #         - this returned value_string value is ready for the  caller to input to hash_string to create the code_id

df[['value_schema', 'value_simple', 'value_jsonb', 'value_string']] = df.apply(lambda row: prepare_dynamic_value_for_sql_issue(row['code'], row['display']), axis=1, result_type='expand')

In [333]:
df

,uuid,code,display,terminology_version_uuid,additional_data,depends_on_property,depends_on_system,depends_on_value,depends_on_display,created_date,value_schema,value_simple,value_jsonb,value_string
0,fb1105a7-c8b9-4461-89cd-5bc9f60d8364,"{[{14040001000004109, Uncontrolled pain (finding), http://snomed.info/sct}, {R52, Uncontrolled pain, urn:oid:2.16.840.1.113883.6.90}], Uncontrolled pain}",Uncontrolled pain,50ed5f2c-7c4c-4586-a7d8-f3665d6380c0,"{'automapping_target': '14040001000004109|Uncontrolled pain (finding)', 'automapping_evidence': 'Condition Code.text is an exact match for SNOMED PT', 'automapping_relationship': 'Equivalent', 'count_of_resources_affected': '1330'}",,,,,2023-03-01 00:00:00+00:00,http://projectronin.io/fhir/StructureDefinition/ronin-conceptMapSourceCodeableConcept,None,"{""coding"":[{""code"":""14040001000004109"",""display"":""Uncontrolled pain (finding)"",""system"":""http://snomed.info/sct""},{""code"":""R52"",""display"":""Uncontrolled pain"",""system"":""urn:oid:2.16.840.1.113883.6.90""}],""text"":""Uncontrolled pain""}","{""coding"":[{""code"":""14040001000004109"",""display"":""Uncontrolled pain (finding)"",""system"":""http://snomed.info/sct""},{""code"":""R52"",""display"":""Uncontrolled pain"",""system"":""urn:oid:2.16.840.1.113883.6.90""}],""text"":""Uncontrolled pain""}"
1,b4646298-292f-4d21-8bd8-f9c0166c1fd8,"{""coding"": [{""system"": ""http://hl7.org/fhir/sid/icd-10-cm"", ""code"": ""R52""}, {""system"": ""http://snomed.info/sct"", ""code"": ""14040001000004109""}], ""text"": ""Uncontrolled pain""}",Uncontrolled pain,50ed5f2c-7c4c-4586-a7d8-f3665d6380c0,None,,,,,2023-12-07 15:10:13.719826+00:00,http://projectronin.io/fhir/StructureDefinition/ronin-conceptMapSourceCodeableConcept,None,"{""coding"":[{""code"":""14040001000004109"",""system"":""http://snomed.info/sct""},{""code"":""R52"",""system"":""http://hl7.org/fhir/sid/icd-10-cm""}],""text"":""Uncontrolled pain""}","{""coding"":[{""code"":""14040001000004109"",""system"":""http://snomed.info/sct""},{""code"":""R52"",""system"":""http://hl7.org/fhir/sid/icd-10-cm""}],""text"":""Uncontrolled pain""}"
2,cd970f63-99bb-47f8-a298-72025f6ede90,"{[{1089991000119109, Oral ulcerative mucositis due to antineoplastic therapy (disorder), http://snomed.info/sct}, {528.01, Ulcerative oral mucositis due to antineoplastic therapy, http://hl7.org/fhir/sid/icd-9-cm/diagnosis}, {K12.31, Ulcerative oral mucositis due to antineoplastic therapy, urn:oid:2.16.840.1.113883.6.90}], Ulcerative oral mucositis due to antineoplastic therapy}",Ulcerative oral mucositis due to antineoplastic therapy,50ed5f2c-7c4c-4586-a7d8-f3665d6380c0,"{'automapping_target': None, 'automapping_evidence': None, 'automapping_relationship': None, 'count_of_resources_affected': '3341'}",,,,,2023-03-01 00:00:00+00:00,http://projectronin.io/fhir/StructureDefinition/ronin-conceptMapSourceCodeableConcept,None,"{""coding"":[{""code"":""K12.31"",""display"":""Ulcerative oral mucositis due to antineoplastic therapy"",""system"":""urn:oid:2.16.840.1.113883.6.90""},{""code"":""1089991000119109"",""display"":""Oral ulcerative mucositis due to antineoplastic therapy (disorder)"",""system"":""http://snomed.info/sct""},{""code"":""528.01"",""display"":""Ulcerative oral mucositis due to antineoplastic therapy"",""system"":""http://hl7.org/fhir/sid/icd-9-cm/diagnosis""}],""text"":""Ulcerative oral mucositis due to antineoplastic therapy""}","{""coding"":[{""code"":""K12.31"",""display"":""Ulcerative oral mucositis due to antineoplastic therapy"",""system"":""urn:oid:2.16.840.1.113883.6.90""},{""code"":""1089991000119109"",""display"":""Oral ulcerative mucositis due to antineoplastic therapy (disorder)"",""system"":""http://snomed.info/sct""},{""code"":""528.01"",""display"":""Ulcerative oral mucositis due to antineoplastic therapy"",""system"":""http://hl7.org/fhir/sid/icd-9-cm/diagnosis""}],""text"":""Ulcerative oral mucositis due to antineoplastic therapy""}"
3,25e0a8d9-e93e-43fc-897c-be05578bdf1e,"{[

In [264]:
# df['is_duplicate'] = df.duplicated(subset=['value_string'], keep=False)

In [295]:
# df.groupby(['is_duplicate']).count()

In [297]:
# df['normalized_code'] = df['value_jsonb'].apply(normalize_source_codeable_concept)

In [334]:
# prepare_json_format_for_sql(json_string: str) -> (str, str, str, str):
"""
Normalizes the JSON string to an expected format with double quotes and no wasted space - calls helper functions
in the correct order: load the object, normalize to RCDM (sorts lists), serialize (sorts keys, strips space)
@return
    code_schema (string) - FHIR primitive name, DataExtensionUrls value, or message prefixed with "format issue:"
    code_simple (string) - old_code if primitive or old_code has "format issue: " - else None
    code_jsonb (string) - None if primitive or old_code has "format issue:" - else, the object with that code_schema
        - note: because this function prepares these values for SQL INSERT query formation, this value is a string -
        all JSON uses double quotes, and if there is a single quote value in the JSON, it is '' for SQL escaping
    code_string (string) - either code_simple, or the code_jsonb binary JSON value correctly serialized to a string
"""
df[['code_schema', 'code_simple', 'code_jsonb', 'code_string']] = df.apply(lambda row: prepare_json_format_for_sql(row['value_string']), axis=1, result_type='expand')


In [335]:
df

,uuid,code,display,terminology_version_uuid,additional_data,depends_on_property,depends_on_system,depends_on_value,depends_on_display,created_date,value_schema,value_simple,value_jsonb,value_string,code_schema,code_simple,code_jsonb,code_string
0,fb1105a7-c8b9-4461-89cd-5bc9f60d8364,"{[{14040001000004109, Uncontrolled pain (finding), http://snomed.info/sct}, {R52, Uncontrolled pain, urn:oid:2.16.840.1.113883.6.90}], Uncontrolled pain}",Uncontrolled pain,50ed5f2c-7c4c-4586-a7d8-f3665d6380c0,"{'automapping_target': '14040001000004109|Uncontrolled pain (finding)', 'automapping_evidence': 'Condition Code.text is an exact match for SNOMED PT', 'automapping_relationship': 'Equivalent', 'count_of_resources_affected': '1330'}",,,,,2023-03-01 00:00:00+00:00,http://projectronin.io/fhir/StructureDefinition/ronin-conceptMapSourceCodeableConcept,None,"{""coding"":[{""code"":""14040001000004109"",""display"":""Uncontrolled pain (finding)"",""system"":""http://snomed.info/sct""},{""code"":""R52"",""display"":""Uncontrolled pain"",""system"":""urn:oid:2.16.840.1.113883.6.90""}],""text"":""Uncontrolled pain""}","{""coding"":[{""code"":""14040001000004109"",""display"":""Uncontrolled pain (finding)"",""system"":""http://snomed.info/sct""},{""code"":""R52"",""display"":""Uncontrolled pain"",""system"":""urn:oid:2.16.840.1.113883.6.90""}],""text"":""Uncontrolled pain""}",http://projectronin.io/fhir/StructureDefinition/ronin-conceptMapSourceCodeableConcept,None,"{""coding"":[{""code"":""14040001000004109"",""display"":""Uncontrolled pain (finding)"",""system"":""http://snomed.info/sct""},{""code"":""R52"",""display"":""Uncontrolled pain"",""system"":""urn:oid:2.16.840.1.113883.6.90""}],""text"":""Uncontrolled pain""}","{""coding"":[{""code"":""14040001000004109"",""display"":""Uncontrolled pain (finding)"",""system"":""http://snomed.info/sct""},{""code"":""R52"",""display"":""Uncontrolled pain"",""system"":""urn:oid:2.16.840.1.113883.6.90""}],""text"":""Uncontrolled pain""}"
1,b4646298-292f-4d21-8bd8-f9c0166c1fd8,"{""coding"": [{""system"": ""http://hl7.org/fhir/sid/icd-10-cm"", ""code"": ""R52""}, {""system"": ""http://snomed.info/sct"", ""code"": ""14040001000004109""}], ""text"": ""Uncontrolled pain""}",Uncontrolled pain,50ed5f2c-7c4c-4586-a7d8-f3665d6380c0,None,,,,,2023-12-07 15:10:13.719826+00:00,http://projectronin.io/fhir/StructureDefinition/ronin-conceptMapSourceCodeableConcept,None,"{""coding"":[{""code"":""14040001000004109"",""system"":""http://snomed.info/sct""},{""code"":""R52"",""system"":""http://hl7.org/fhir/sid/icd-10-cm""}],""text"":""Uncontrolled pain""}","{""coding"":[{""code"":""14040001000004109"",""system"":""http://snomed.info/sct""},{""code"":""R52"",""system"":""http://hl7.org/fhir/sid/icd-10-cm""}],""text"":""Uncontrolled pain""}",http://projectronin.io/fhir/StructureDefinition/ronin-conceptMapSourceCodeableConcept,None,"{""coding"":[{""code"":""14040001000004109"",""system"":""http://snomed.info/sct""},{""code"":""R52"",""system"":""http://hl7.org/fhir/sid/icd-10-cm""}],""text"":""Uncontrolled pain""}","{""coding"":[{""code"":""14040001000004109"",""system"":""http://snomed.info/sct""},{""code"":""R52"",""system"":""http://hl7.org/fhir/sid/icd-10-cm""}],""text"":""Uncontrolled pain""}"
2,cd970f63-99bb-47f8-a298-72025f6ede90,"{[{1089991000119109, Oral ulcerative mucositis due to antineoplastic therapy (disorder), http://snomed.info/sct}, {528.01, Ulcerative oral mucositis due to antineoplastic therapy, http://hl7.org/fhir/sid/icd-9-cm/diagnosis}, {K12.31, Ulcerative oral mucositis due to antineoplastic therapy, urn:oid:2.16.840.1.113883.6.90}], Ulcerative oral mucositis due to antineoplastic therapy}",Ulcerative oral mucositis due to antineoplastic therapy,50ed5f2c-7c4c-4586-a7d8-f3665d6380c0,"{'automapping_target': None, 'automapping_evidence': None, 'automapping_relationship': None, 'count_of_resources_affected': '3341'}",,,,,2023-03-01 00:00:00+00:00,http://projectronin.io/fhir/StructureDefinition/ronin-conceptMapSour

In [268]:
# df['is_duplicate'] = df.duplicated(subset=['code_jsonb'], keep=False)

In [269]:
# df.groupby(['is_duplicate']).count()

,uuid,code,display,terminology_version_uuid,additional_data,depends_on_property,depends_on_system,depends_on_value,depends_on_display,created_date,value_schema,value_simple,value_jsonb,value_string,code_schema,code_simple,code_jsonb,code_string
is_duplicate,,,,,,,,,,,,,,,,,,
False,65806,65806,65806,65806,65640,65806,65806,65806,65806,65806,65806,0,65806,65806,65806,0,65806,65806


In [196]:
df_duplicates = df[df['is_duplicate']]


In [197]:
df_duplicates.head()

,uuid,code,display,terminology_version_uuid,additional_data,depends_on_property,depends_on_system,depends_on_value,depends_on_display,created_date,value_schema,value_simple,value_jsonb,value_string,is_duplicate,code_schema,code_simple,code_jsonb,code_string
0,022316dd-e134-4938-ab62-fd0915b2b589,"{[{D68.0, null, urn:oid:2.16.840.1.113883.6.90...",von Willebrand's disease,54589835-c44a-4392-8bc7-67b396bb7b4a,{'automapping_target': '128105004|von Willebra...,,,,,2023-03-01 00:00:00+00:00,http://projectronin.io/fhir/StructureDefinitio...,None,"{""coding"":[{""code"":""D68.0"",""system"":""urn:oid:2...","{""coding"":[{""code"":""D68.0"",""system"":""urn:oid:2...",True,http://projectronin.io/fhir/StructureDefinitio...,None,"{""coding"":[{""code"":""D68.0"",""system"":""urn:oid:2...","{""coding"":[{""code"":""D68.0"",""system"":""urn:oid:2..."
1,87246277-73e0-4e47-b94c-ccb237780d62,"{[{128105004, von Willebrand disorder (disorde...",von Willebrand's disease,54589835-c44a-4392-8bc7-67b396bb7b4a,{'automapping_target': '128105004|von Willebra...,,,,,2023-03-01 00:00:00+00:00,http://projectronin.io/fhir/StructureDefinitio...,None,"{""coding"":[{""code"":""286.4"",""display"":""von Will...","{""coding"":[{""code"":""286.4"",""display"":""von Will...",True,http://projectronin.io/fhir/StructureDefinitio...,None,"{""coding"":[{""code"":""286.4"",""display"":""von Will...","{""coding"":[{""code"":""286.4"",""display"":""von Will..."
2,d0abd751-08bb-4305-bc9c-24db210c0328,"{[{128105004, von Willebrand disorder (disorde...",von Willebrand disorder <Unspecified von Wille...,54589835-c44a-4392-8bc7-67b396bb7b4a,"{'automapping_target': None, 'automapping_evid...",,,,,2023-03-01 00:00:00+00:00,http://projectronin.io/fhir/StructureDefinitio...,None,"{""coding"":[{""code"":""D68.00"",""display"":""von Wil...","{""coding"":[{""code"":""D68.00"",""display"":""von Wil...",True,http://projectronin.io/fhir/StructureDefinitio...,None,"{""coding"":[{""code"":""D68.00"",""display"":""von Wil...","{""coding"":[{""code"":""D68.00"",""display"":""von Wil..."
3,4cf26e25-c625-4f73-8b26-b92ba42ee74f,"{[{D68.0, null, urn:oid:2.16.840.1.113883.6.90...",von Willebrand disorder,54589835-c44a-4392-8bc7-67b396bb7b4a,{'automapping_target': '128105004|von Willebra...,,,,,2023-03-01 00:00:00+00:00,http://projectronin.io/fhir/StructureDefinitio...,None,"{""coding"":[{""code"":""D68.0"",""system"":""urn:oid:2...","{""coding"":[{""code"":""D68.0"",""system"":""urn:oid:2...",True,http://projectronin.io/fhir/StructureDefinitio...,None,"{""coding"":[{""code"":""D68.0"",""system"":""urn:oid:2...","{""coding"":[{""code"":""D68.0"",""system"":""urn:oid:2..."
4,e45ec6fa-309d-4f91-9587-8873a3cfce04,"{[{D68.00, null, urn:oid:2.16.840.1.113883.6.9...",von Willebrand disorder,54589835-c44a-4392-8bc7-67b396bb7b4a,{'automapping_target': '128105004|von Willebra...,,,,,2023-03-01 00:00:00+00:00,http://projectronin.io/fhir/StructureDefinitio...,None,"{""coding"":[{""code"":""128105004"",""system"":""urn:o...","{""coding"":[{""code"":""128105004"",""system"":""urn:o...",True,http://projectronin.io/fhir/StructureDefinitio...,None,"{""coding"":[{""code"":""128105004"",""system"":""urn:o...","{""coding"":[{""code"":""128105004"",""system"":""urn:o..."


In [193]:
df.head()

,uuid,code,display,terminology_version_uuid,additional_data,depends_on_property,depends_on_system,depends_on_value,depends_on_display,created_date,value_schema,value_simple,value_jsonb,value_string,is_duplicate,code_schema,code_simple,code_jsonb,code_string
0,022316dd-e134-4938-ab62-fd0915b2b589,"{[{D68.0, null, urn:oid:2.16.840.1.113883.6.90...",von Willebrand's disease,54589835-c44a-4392-8bc7-67b396bb7b4a,{'automapping_target': '128105004|von Willebra...,,,,,2023-03-01 00:00:00+00:00,http://projectronin.io/fhir/StructureDefinitio...,None,"{""coding"":[{""code"":""D68.0"",""system"":""urn:oid:2...","{""coding"":[{""code"":""D68.0"",""system"":""urn:oid:2...",False,http://projectronin.io/fhir/StructureDefinitio...,None,"{""coding"":[{""code"":""D68.0"",""system"":""urn:oid:2...","{""coding"":[{""code"":""D68.0"",""system"":""urn:oid:2..."
1,87246277-73e0-4e47-b94c-ccb237780d62,"{[{128105004, von Willebrand disorder (disorde...",von Willebrand's disease,54589835-c44a-4392-8bc7-67b396bb7b4a,{'automapping_target': '128105004|von Willebra...,,,,,2023-03-01 00:00:00+00:00,http://projectronin.io/fhir/StructureDefinitio...,None,"{""coding"":[{""code"":""286.4"",""display"":""von Will...","{""coding"":[{""code"":""286.4"",""display"":""von Will...",False,http://projectronin.io/fhir/StructureDefinitio...,None,"{""coding"":[{""code"":""286.4"",""display"":""von Will...","{""coding"":[{""code"":""286.4"",""display"":""von Will..."
2,d0abd751-08bb-4305-bc9c-24db210c0328,"{[{128105004, von Willebrand disorder (disorde...",von Willebrand disorder <Unspecified von Wille...,54589835-c44a-4392-8bc7-67b396bb7b4a,"{'automapping_target': None, 'automapping_evid...",,,,,2023-03-01 00:00:00+00:00,http://projectronin.io/fhir/StructureDefinitio...,None,"{""coding"":[{""code"":""D68.00"",""display"":""von Wil...","{""coding"":[{""code"":""D68.00"",""display"":""von Wil...",False,http://projectronin.io/fhir/StructureDefinitio...,None,"{""coding"":[{""code"":""D68.00"",""display"":""von Wil...","{""coding"":[{""code"":""D68.00"",""display"":""von Wil..."
3,4cf26e25-c625-4f73-8b26-b92ba42ee74f,"{[{D68.0, null, urn:oid:2.16.840.1.113883.6.90...",von Willebrand disorder,54589835-c44a-4392-8bc7-67b396bb7b4a,{'automapping_target': '128105004|von Willebra...,,,,,2023-03-01 00:00:00+00:00,http://projectronin.io/fhir/StructureDefinitio...,None,"{""coding"":[{""code"":""D68.0"",""system"":""urn:oid:2...","{""coding"":[{""code"":""D68.0"",""system"":""urn:oid:2...",False,http://projectronin.io/fhir/StructureDefinitio...,None,"{""coding"":[{""code"":""D68.0"",""system"":""urn:oid:2...","{""coding"":[{""code"":""D68.0"",""system"":""urn:oid:2..."
4,e45ec6fa-309d-4f91-9587-8873a3cfce04,"{[{D68.00, null, urn:oid:2.16.840.1.113883.6.9...",von Willebrand disorder,54589835-c44a-4392-8bc7-67b396bb7b4a,{'automapping_target': '128105004|von Willebra...,,,,,2023-03-01 00:00:00+00:00,http://projectronin.io/fhir/StructureDefinitio...,None,"{""coding"":[{""code"":""128105004"",""system"":""urn:o...","{""coding"":[{""code"":""128105004"",""system"":""urn:o...",False,http://projectronin.io/fhir/StructureDefinitio...,None,"{""coding"":[{""code"":""128105004"",""system"":""urn:o...","{""coding"":[{""code"":""128105004"",""system"":""urn:o..."


In [336]:
def normalize_json(json_str):
    try:
        data = json.loads(json_str)
        coding = data.get('coding', [])

        # Process each entry in 'coding'
        for entry in coding:
            entry['code'] = entry['code'].strip()
            entry['system'] = entry['system'].strip()
            # If 'display' key is None or doesn't exist, remove it
            if 'display' in entry and entry['display'] is None:
                del entry['display']
            elif 'display' in entry:
                entry['display'] = entry['display'].strip()

        # Sort the dictionaries within the list by 'code'
        sorted_coding = sorted(coding, key=lambda x: x['code'] + x['system'])

        # Strip whitespace from 'text'
        text = data.get('text', '').strip()

        # Return a JSON string of the sorted list of dictionaries with 'display' key removed if it's null
        return json.dumps({'coding': sorted_coding, 'text': text}, sort_keys=True)
    except json.JSONDecodeError:
        return json_str  # or return None to indicate an error


In [337]:
# Apply the function to the 'code' column to normalize the data
df['normalized_code'] = df['code_string'].apply(normalize_json)


In [338]:
# hash_string(input_string: str) -> str:
df['hash_id'] = df['normalized_code'].apply(hash_string)

In [272]:
df.head()

,uuid,code,display,terminology_version_uuid,additional_data,depends_on_property,depends_on_system,depends_on_value,depends_on_display,created_date,value_schema,value_simple,value_jsonb,value_string,is_duplicate,code_schema,code_simple,code_jsonb,code_string,normalized_code
0,87246277-73e0-4e47-b94c-ccb237780d62,"{[{128105004, von Willebrand disorder (disorder), http://snomed.info/sct}, {286.4, von Willebrand's disease, http://hl7.org/fhir/sid/icd-9-cm/diagnosis}, {D68.0, von Willebrand's disease, urn:oid:2.16.840.1.113883.6.90}], von Willebrand's disease}",von Willebrand's disease,54589835-c44a-4392-8bc7-67b396bb7b4a,"{'automapping_target': '128105004|von Willebrand disorder (disorder)', 'automapping_evidence': 'Condition Code.text matches one or many acceptable synonyms of the concept. Synonyms are: Factor VIII deficiency with vascular defect, vWD - von Willebrand's disease, von Willebrand disease, platelet type, Vascular hemophilia, Pseudohemophilia type B, Angiohemophilia, Constitutional thrombopathy, von Willebrand's disease, von Willebrand disease, von Willebrand-Jurgens disease', 'automapping_relationship': 'Equivalent', 'count_of_resources_affected': '7'}",,,,,2023-03-01 00:00:00+00:00,http://projectronin.io/fhir/StructureDefinition/ronin-conceptMapSourceCodeableConcept,None,"{""coding"":[{""code"":""286.4"",""display"":""von Willebrand''s disease"",""system"":""http://hl7.org/fhir/sid/icd-9-cm/diagnosis""},{""code"":""128105004"",""display"":""von Willebrand disorder (disorder)"",""system"":""http://snomed.info/sct""},{""code"":""D68.0"",""display"":""von Willebrand''s disease"",""system"":""urn:oid:2.16.840.1.113883.6.90""}],""text"":""von Willebrand''s disease""}","{""coding"":[{""code"":""286.4"",""display"":""von Willebrand's disease"",""system"":""http://hl7.org/fhir/sid/icd-9-cm/diagnosis""},{""code"":""128105004"",""display"":""von Willebrand disorder (disorder)"",""system"":""http://snomed.info/sct""},{""code"":""D68.0"",""display"":""von Willebrand's disease"",""system"":""urn:oid:2.16.840.1.113883.6.90""}],""text"":""von Willebrand's disease""}",False,http://projectronin.io/fhir/StructureDefinition/ronin-conceptMapSourceCodeableConcept,None,"{""coding"":[{""code"":""286.4"",""display"":""von Willebrand''s disease"",""system"":""http://hl7.org/fhir/sid/icd-9-cm/diagnosis""},{""code"":""128105004"",""display"":""von Willebrand disorder (disorder)"",""system"":""http://snomed.info/sct""},{""code"":""D68.0"",""display"":""von Willebrand''s disease"",""system"":""urn:oid:2.16.840.1.113883.6.90""}],""text"":""von Willebrand''s disease""}","{""coding"":[{""code"":""286.4"",""display"":""von Willebrand's disease"",""system"":""http://hl7.org/fhir/sid/icd-9-cm/diagnosis""},{""code"":""128105004"",""display"":""von Willebrand disorder (disorder)"",""system"":""http://snomed.info/sct""},{""code"":""D68.0"",""display"":""von Willebrand's disease"",""system"":""urn:oid:2.16.840.1.113883.6.90""}],""text"":""von Willebrand's disease""}","{""coding"": [{""code"": ""128105004"", ""display"": ""von Willebrand disorder (disorder)"", ""system"": ""http://snomed.info/sct""}, {""code"": ""286.4"", ""display"": ""von Willebrand's disease"", ""system"": ""http://hl7.org/fhir/sid/icd-9-cm/diagnosis""}, {""code"": ""D68.0"", ""display"": ""von Willebrand's disease"", ""system"": ""urn:oid:2.16.840.1.113883.6.90""}], ""text"": ""von Willebrand's disease""}"
1,022316dd-e134-4938-ab62-fd0915b2b589,"{[{D68.0, null, urn:oid:2.16.840.1.113883.6.90}, {128105004, null, urn:oid:2.16.840.1.113883.6.96}], von Willebrand's disease}",von Willebrand's disease,54589835-c44a-4392-8bc7-67b396bb7b4a,"{'automapping_target': '128105004|von Willebrand disorder (disorder)', 'automapping_evidence': 'Condition Code.text matches one or many acceptable synonyms of the concept. Synonyms are: Factor VIII deficiency with vascular defect, vWD - von Willebrand's disease, von Willebrand disease, platelet type, Vascular hemophilia, Pseudohemophilia type B, Angiohemop

In [273]:
# Now, you can identify duplicates based on the 'normalized_code' column
df['is_duplicate'] = df.duplicated('normalized_code', keep=False)

In [274]:
df.groupby(['is_duplicate']).count()

,uuid,code,display,terminology_version_uuid,additional_data,depends_on_property,depends_on_system,depends_on_value,depends_on_display,created_date,value_schema,value_simple,value_jsonb,value_string,code_schema,code_simple,code_jsonb,code_string,normalized_code
is_duplicate,,,,,,,,,,,,,,,,,,,
False,65806,65806,65806,65806,65640,65806,65806,65806,65806,65806,65806,0,65806,65806,65806,0,65806,65806,65806


In [344]:
df.to_csv("/Users/jonbarton/Downloads/mda_duplicates_quest.csv")

In [343]:
# for index, value in df['normalized_code'].head().items():
#     print(f"Row {index}:")
#     print(value)
#     print()

In [339]:
df['hash_id'].head

<bound method NDFrame.head of 0      cf14efe91182b3558c3a04b6dbaefeb1
1      31ed9f71122023f6bfc2a89f3be1460a
2      5a53d2fbd509ffb4d42eb9c4a60b52e1
3      8b79bb76de020f2ef6d39ae5d633a0ab
4      490174c760743208993283088fc800a7
                     ...               
420    7d7989c6504b195f9ca1b038fc418144
421    d7092e3c08bf424929d9ef2837054b42
422    e93f59cf3ddd94d020f1d34ad4f76719
423    54c4e28a77e2a32f7c3105529f20e8f9
424    8e870a8d027af6786736c757b1bb7b02
Name: hash_id, Length: 425, dtype: object>

In [229]:
df['duplicates'] = df.duplicated('hash_id', keep=False)


In [230]:
df.groupby(['duplicates']).count()


,uuid,code,display,terminology_version_uuid,additional_data,depends_on_property,depends_on_system,depends_on_value,depends_on_display,created_date,...,value_jsonb,value_string,is_duplicate,code_schema,code_simple,code_jsonb,code_string,normalized_code,duplicated,hash_id
duplicates,,,,,,,,,,,,,,,,,,,,,
False,65805,65805,65805,65805,65639,65805,65805,65805,65805,65805,...,65805,65805,65805,65805,0,65805,65805,65805,65805,65805
True,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,0,4,4,4,4,4


In [340]:
# Group by the hash and count the occurrences
hash_groups = df.groupby('hash_id').size()

# Filter groups with more than one occurrence to find duplicates
duplicate_hashes = hash_groups[hash_groups > 1].index

# Now, select rows where the hash is in the list of duplicates
df_duplicates = df[df['hash_id'].isin(duplicate_hashes)]

In [341]:
df_duplicates.head()

,uuid,code,display,terminology_version_uuid,additional_data,depends_on_property,depends_on_system,depends_on_value,depends_on_display,created_date,value_schema,value_simple,value_jsonb,value_string,code_schema,code_simple,code_jsonb,code_string,normalized_code,hash_id


In [279]:
df['null_removed_hash_id'] = df['normalized_code'].apply(hash_string)


In [280]:
# Group by the hash and count the occurrences
hash_groups = df.groupby('null_removed_hash_id').size()

# Filter groups with more than one occurrence to find duplicates
duplicate_hashes = hash_groups[hash_groups > 1].index

# Now, select rows where the hash is in the list of duplicates
new_df_duplicates = df[df['null_removed_hash_id'].isin(duplicate_hashes)]

In [281]:
df['is_duplicate'] = df.duplicated('normalized_code', keep=False)
df_duplicates = df[df['is_duplicate']]

In [282]:
df_duplicates.head()

,uuid,code,display,terminology_version_uuid,additional_data,depends_on_property,depends_on_system,depends_on_value,depends_on_display,created_date,...,value_jsonb,value_string,is_duplicate,code_schema,code_simple,code_jsonb,code_string,normalized_code,hash_id,null_removed_hash_id
